# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Neumann  ->  J. Neumann  |  ['J. Neumann']
J. Li  ->  J. Li  |  ['J. Li']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 55 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2312.13355


extracting tarball to tmp_2312.13355...

 done.


list index out of range
Retrieving document from  https://arxiv.org/e-print/2312.13612


extracting tarball to tmp_2312.13612...

 done.
Retrieving document from  https://arxiv.org/e-print/2312.13684


extracting tarball to tmp_2312.13684...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.13355-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.13355) | **The universal variability of the stellar initial mass function probed by  the TIMER survey**  |
|| I. Martín-Navarro, et al. -- incl., <mark>J. Neumann</mark> |
|*Appeared on*| *2023-12-22*|
|*Comments*| *16 pages, 11 figures. Accepted for publication in Astronomy and Astrophysics*|
|**Abstract**| The debate about the universality of the stellar initial mass function (IMF) revolves around two competing lines of evidence. While measurements in the Milky Way, an archetypal spiral galaxy, seem to support an invariant IMF, the observed properties of massive early-type galaxies (ETGs) favor an IMF somehow sensitive to the local star formation conditions. The fundamental methodological and physical differences between both approaches have hampered, however, a comprehensive understanding of IMF variations. We describe here an improved modelling scheme that allows for the first time consistent IMF measurements across stellar populations with different ages and complex star formation histories. Making use of the exquisite MUSE optical data from the TIMER survey and powered by the MILES stellar population models, we show the age, metallicity, [Mg/Fe], and IMF slope maps of the inner regions of NGC 3351, a spiral galaxy with a mass similar to that of the Milky Way. The measured IMF values in NGC3351 follow the expectations from a Milky Way-like IMF, although they simultaneously show systematic and spatially coherent variations, particularly for low-mass stars. In addition, our stellar population analysis reveals the presence of metal-poor and Mg-enhanced star-forming regions that appear to be predominantly enriched by the stellar ejecta of core-collapse supernovae. Our findings showcase therefore the potential of detailed studies of young stellar populations to better understand the early stages of galaxy evolution and, in particular, the origin of the observed IMF variations beyond and within the Milky Way. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.13612-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.13612) | **A seven-Earth-radius helium-burning star inside a 20.5-min detached  binary**  |
|| J. Lin, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-12-22*|
|*Comments*| *24 pages, 11 figures, 1 table*|
|**Abstract**| Binary evolution theory predicts that the second common envelope (CE) ejection can produce low-mass (0.32-0.36 Msun) subdwarf B (sdB) stars inside ultrashort-orbital-period binary systems, as their helium cores are ignited under nondegenerate conditions. With the orbital decay driven by gravitational-wave (GW) radiation, the minimum orbital periods of detached sdB binaries could be as short as ~20 minutes. However, only four sdB binaries with orbital periods below an hour have been reported so far, while none of them has an orbital period approaching the above theoretical limit. Here we report the discovery of a 20.5-minute-orbital-period ellipsoidal binary, TMTS J052610.43+593445.1, in which the visible star is being tidally deformed by an invisible carbon-oxygen white dwarf (WD) companion. The visible component is inferred to be an sdB star with a mass of ~0.33 Msun, approaching that of helium-ignition limit, although a He-core WD cannot be completely ruled out. In particular, the radius of this low-mass sdB star is only 0.066 Rsun, about seven Earth radii, possibly representing the most compact nondegenerate star ever known. Such a system provides a key clue to map the binary evolution scheme from the second CE ejection to the formation of AM CVn stars having a helium-star donor, and it will also serve as a crucial verification binary of space-borne GW detectors in the future. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.13684-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.13684) | **Satellite Impact on Astronomical Observations Based on Elliptical Orbit  Model**  |
|| T. Hu, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-12-22*|
|*Comments*| *9 pages, 7 figures, submitted to A&A, any comments are welcome*|
|**Abstract**| Space-based and ground-based telescopes have extensively documented the impact of satellites on astronomical observations. With the proliferation of satellite mega-constellation programs, their influence on astronomical observations has become undeniable. It is crucial to quantify the impact of satellites on telescopes. To address this need, we have enhanced the circular orbit model for satellites and introduced a methodology based on two-line element (TLE) orbit data. This involves constructing a satellite probability distribution model to evaluate the impact of satellites on telescopes. Using our method, we assessed the satellite impact on global observatories. The results indicate that the regions most severely affected by satellite interference currently are those near the equator, with latitudes around 50 and 80 degrees experiencing the most significant impact from low Earth orbit satellites. Furthermore, we validated the reliability of our method using imaging data obtained from the focal surface acquisition camera of the LAMOST telescope. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2312.13355.md
    + _build/html/tmp_2312.13355/./figures/all_IMF.png
    + _build/html/tmp_2312.13355/./figures/OffGrid_light.png
    + _build/html/tmp_2312.13355/./figures/OffGrid_mass.png
    + _build/html/tmp_2312.13355/./figures/TiO2_shape.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\msun}{\hbox{M_{\odot}}}$
$\newcommand{\kms}{\hbox{km s^{-1}}}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# The universal variability of the stellar initial mass function probed by the TIMER survey

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2312.13355-b31b1b.svg)](https://arxiv.org/abs/2312.13355)<mark>Appeared on: 2023-12-22</mark> -  _16 pages, 11 figures. Accepted for publication in Astronomy and Astrophysics_

</div>
<div id="authors">

I. Martín-Navarro, et al. -- incl., <mark>J. Neumann</mark>

</div>
<div id="abstract">

**Abstract:** The debate about the universality of the stellar initial mass function (IMF) revolves around two competing lines of evidence. While measurements in the Milky Way, an archetypal spiral galaxy, seem to support an invariant IMF, the observed properties of massive early-type galaxies (ETGs) favor an IMF somehow sensitive to the local star formation conditions. The fundamental methodological and physical differences between both approaches have hampered, however, a comprehensive understanding of IMF variations. We describe here an improved modelling scheme that allows for the first time consistent IMF measurements across stellar populations with different ages and complex star formation histories. Making use of the exquisite MUSE optical data from the TIMER survey and powered by the MILES stellar population models, we show the age, metallicity, [ Mg/Fe ] , and IMF slope maps of the inner regions of NGC 3351, a spiral galaxy with a mass similar to that of the Milky Way. The measured IMF values in NGC 3351 follow the expectations from a Milky Way-like IMF, although they simultaneously show systematic and spatially coherent variations, particularly for low-mass stars. In addition, our stellar population analysis reveals the presence of metal-poor and Mg-enhanced star-forming regions that appear to be predominantly enriched by the stellar ejecta of core-collapse supernovae. Our findings showcase therefore the potential of detailed studies of young stellar populations to better understand the early stages of galaxy evolution and,  in particular, the origin of the observed IMF variations beyond and within the Milky Way.

</div>

<div id="div_fig1">

<img src="tmp_2312.13355/./figures/all_IMF.png" alt="Fig10" width="100%"/>

**Figure 10. -** Line strength age and IMF sensitivity. From left to right and top to bottom we show the age dependence of the Mgb, H$_{\beta_O}$, Nad, and $TiO_2$ spectral features, respectively. Solid lines correspond to the MILES $\alpha$-variable models and dashed ones indicate their (adapted) super-young extension. The gray shaded area in the background shows the unmodified super-young model predictions for a Milky Way-like IMF (see main text for more details). Orange lines are predictions for a bottom-heavy IMF (i.e. steeper than the Milky Way standard), blue lines for a top-heavy (i.e. flatter) IMF, and black lines for the reference Salpeter IMF. The rapid increase in the equivalent width of the indices (but H$_{\beta_O}$) at ages around $\sim 10$ Myr is caused by the sudden appearance of red supergiant stars. Predictions are shown at the native resolution of the MILES models (2.51 Å). (*fig:indices*)

</div>
<div id="div_fig2">

<img src="tmp_2312.13355/./figures/OffGrid_light.png" alt="Fig11.1" width="50%"/><img src="tmp_2312.13355/./figures/OffGrid_mass.png" alt="Fig11.2" width="50%"/>

**Figure 11. -** H$_{\beta_o}$ vs NaD index-index diagram. Gray grids show the MILES predictions at the native resolution of the models (2.51 Å) and solar metallicity for different ages and IMF slopes, as indicated by the labels. Blue solid lines highlight the age variation for a Salpeter IMF. Dashed blue lines and filled circles track the range of expected values for a composite stellar population of two SSP models (14 Gyr and 0.35 Gyr), both of them with a Milky Way-like IMF. Symbols are color-coded by the light (left panel) and mass (right panel) fraction associated to the young SSP model. This simplified yet realistic example shows how the best-fitting SSP solution fails to recover the true value of $\Gamma=1.3$ when applied to composite stellar populations, which could result in unrealistically high IMF slope values ($\Gamma\sim 3.3$). (*fig:offgrid*)

</div>
<div id="div_fig3">

<img src="tmp_2312.13355/./figures/TiO2_shape.png" alt="Fig2" width="100%"/>

**Figure 2. -** Optical sensitivity to the shape of the IMF. Black, orange, and purple lines indicate the age sensitivity of the $TiO_2$ absorption feature for a Salpeter IMF, a bottom-heavy single power law IMF, and a bottom-heavy broken power law IMF, respectively. For old stellar populations, as those harbored by massive quiescent galaxies, both single power law and broken power law IMF parametrizations are indistinguishable since the stellar mass range probed along the IMF is rather narrow. For young stellar populations however, different model assumptions on the IMF shape lead to dramatically different model predictions, highlighting the potential use of line-strength indices to actually constrain the shape of the IMF. Predictions are shown at the native resolution of the MILES models (2.51 Å). (*fig:shape*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2312.13355"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

347  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
